In [ ]:
function Economy(;σ = 2.0,        #risk aversion
                  β = 0.85,       #discount factor 
                  ψ = 1/2.22,     #determines labor elasticity
                  μ = 1/3,        #probability of re-entry into credit markets 
                  π = 0.5,        #weight in utility for government expenditure
                  rf = 0.010,     #risk free interest rate
                  SDFC = 1,       #equals to "1" if we have a stochastic discount factor as in Hatchondo, Martinez and Roch (2012)
                  ξ = 0.7,        #proportion of the bond coupon that is "fixed payment", e.g. (1-ξ) is the share of the coupon payment indexed to the commodity price
                  γ = 0.05,       #parameter that determines how indexed portion of coupon payment is determined by commodity price, in other words its "sensitivity"
                  α = 0.65,       #additional parameter for stochastic discount factor (I modified the sensitivity to the productivity shock)
                  ϑ = 0.256,      #normalizes the SDFC so that the max(SDFC) = 1  
                  ϕ = 0.99,       #penalty for default (Cuadra et al.: 0.99)
                  ω = 0.051,      #level parameter for TFP loss under default
        
                  ρshock = 0.597,  #persistence parameter for productivity shocks (Cuadra et al.: 0.85 quarterly calibration)
                  σshock = 0.0064, #volatility parameter for productivity shocks (Cuadra et al.: 0.006 quarterly calibration)
                  Ny = 5,          #size of grid for productivity shocks
        
                  # θ = 0.074,       #share of oil revenue in GDP (it is 0.09 in Pieschacon 2012, Table 2, pg. 260)
                          
                  toldiffvalue  = 0.001, #tolerance for value function iteration 
                  
                  # HPlambda = 100, #Hodrick-Prescott filter parameter (1600 for quarterly data, 100 for annual data).
                  FORWD = 0, #Check qué es
                  λ = ρshock, 
                  σEps = σshock,
                  
                  MMM = 1.95,      
                  #BurnTm = 100, #number of periods that are burnt
                  #Tsim = BurnTm + 250*4, #number of periods for each simulation
                  #Nsim = 1000, #number of simulations , 
                  Nb = 100,
                  b_inf = -0.2343, 
                  b_sup = 1.,
                  ρ = 0.66, 
                  std_ϵ = 0.034,
                  NNN = Ny, #gridshock,
                  τ = 0.2, 
                  α0 = 11, 
                  δ = -141,
                  Δ = 0.25,
                  Nbs = 20, 
                  bs_inf = 0.0,
                  bs_sup = 0.5,
                  qs=1/rf-1, 
        
                  ρcop = 0.94, 
                  σcop = 0.23, 
                  Nc = 5, 
                  z = 0.074, #share of cop revenue in GDP
                  crash = 0.35,    #probability of a sharp copper drop
                  restriccion = 0, #Tipo de regla, 0=modelo óptimo. 
                  y_mean = 0., #Ingreso no cobre de largo plazo 
                  x_mean = 0., #Ingreso cobre de largo plazo
                  Θ      = 0., #Regla de balance estructural de largo plazo
                  b_limit = -0.05, #Límite deuda bruta sobre PIB 
                  Bs_mean = 0. , #Fondos soberanos de largo plazo
                  Ni = 11, #Dimensión para probabilidad de default
        
        
        
  ) 
    
    #Grids
    
    grid_yc = Array{Float64}(undef, Ny) #Output in credit 
    grid_yd = Array{Float64}(undef, Ny) #Output in default

    grid_b = collect(range(b_inf, b_sup, length = Nb)) #Long Term grid: grid_b (state variable) and tax rates
    
    if Nbs == 1
        grid_bs = bs_inf
    else
        grid_bs = collect(range(bs_inf, bs_sup, length = Nbs))    #Short Term grid: grid_short (state variable) and tax rates    
    end
    
    grid_c = Array{Float64}(undef, Nc) #Copper

    #Matrix
    g_c = Array{Float64}(undef, Nb, Ny, Nbs, Nc)
    g_d = Array{Float64}(undef, Ny, Nbs, Nc)
    l_c = Array{Float64}(undef, Ny)
    l_d = Array{Float64}(undef, Ny)    
    c_c = Array{Float64}(undef, Ny)#Array{Float64}(undef, Nb, Ny, Nbs, Nc)
    c_d = Array{Float64}(undef, Ny)#Array{Float64}(undef, Ny, Nbs, Nc)    
    
    d = Array{Float64}(undef, Nb, Ny, Nbs, Nc)

    Vc = (-113)*ones(Nb, Ny, Nbs, Nc)
    Vd = (-113)*ones(Nb, Ny, Nbs, Nc)
    V_bar_d = Array{Float64}(undef, Ny, Nbs, Nc)
    V_bar = Array{Float64}(undef, Nb, Ny, Nbs, Nc)
    Vd0 = Array{Float64}(undef, Ny, Nbs, Nc)
    
    vd   = Array{Float64}(undef, Nbs) #CHECK
    vc = Array{Float64}(undef, Nb, Nbs) #CHECK
    Vd_dim = ones(Nb, Nbs) #CHECK
    
    b_c = Array{Float64}(undef, Nb, Ny, Nbs, Nc) 
    b_d = Array{Float64}(undef, Nb, Ny, Nbs, Nc)
    bs_c = Array{Float64}(undef, Nb, Ny, Nbs, Nc) 
    bs_d = Array{Float64}(undef, Nb, Ny, Nbs, Nc)
    
    y_c_full = Array{Float64}(undef, Ny, Nc) 
    y_d_full = Array{Float64}(undef, Ny, Nc)    
    
    q = Array{Float64}(undef, Nb, Ny, Nbs, Nc)  
    prob_def = Array{Float64}(undef, Nb, Ny, Nbs, Nc)      

    #Index
    index_b_c = zeros(Int, Nb, Ny, Nbs, Nc)
    index_b_d = zeros(Int, Nb, Ny, Nbs, Nc)
    index_bs_c = zeros(Int, Nb, Ny, Nbs, Nc)
    index_bs_d = zeros(Int, Nb, Ny, Nbs, Nc)  
    
    #Tauchen
    newΛ = Array{Float64}(undef, NNN, NNN)
    Λ = Array{Float64}(undef, NNN, NNN)    
    midshockcop = Int((size(Array{Float64}(undef, NNN, NNN))[2]+1)/2)

    newΓ = Array{Float64}(undef, Nc, Nc)
    Γ = Array{Float64}(undef, Nc, Nc)
    
    #SDF
    M = Array{Float64}(undef, Ny, Ny) 
    
   
    return (σ=σ, β=β, ψ=ψ, μ=μ, π=π, rf=rf, SDFC=SDFC, ξ=ξ, γ=γ, α=α, ϑ=ϑ, ϕ=ϕ, ω=ω, 
        ρshock=ρshock, σshock=σshock,
        crash=crash, toldiffvalue=toldiffvalue, σEps=σEps, NNN=NNN, MMM=MMM,
        FORWD=FORWD, Nb=Nb,  d=d, Vc=Vc, Vd=Vd,  g_c=g_c, g_d=g_d,
        b_c=b_c, b_d=b_d,bs_c=bs_c, bs_d=bs_d, λ=λ, V_bar=V_bar,V_bar_d=V_bar_d, vd=vd, Vd0=Vd0, 
        Vd_dim=Vd_dim, vc=vc, newΛ=newΛ, Λ=Λ, #midshock=midshock bc_grid=bc_grid,
        M=M, midshockcop=midshockcop, index_b_c=index_b_c,  #bd_grid=bd_grid,
        index_b_d=index_b_d, #BurnTm=BurnTm, Tsim=Tsim, Nsim=Nsim,
        b_limit=b_limit, b_inf=b_inf, b_sup=b_sup, 
        Nbs=Nbs,bs_inf=bs_inf,bs_sup=bs_sup,q=q,qs=qs,
        grid_b=grid_b, grid_bs=grid_bs,index_bs_c=index_bs_c,index_bs_d=index_bs_d,
        Ny=Ny,grid_yc=grid_yc, grid_yd=grid_yd, τ=τ, α0=α0, δ=δ, z=z, Δ=Δ, prob_def=prob_def, 
        newΓ=newΓ, Γ=Γ, grid_c=grid_c, Nc=Nc, ρcop=ρcop, σcop=σcop, 
        l_c=l_c, l_d=l_d, restriccion=restriccion, y_mean=y_mean, x_mean=x_mean, Θ=Θ, 
        Bs_mean=Bs_mean, Ni=Ni, y_c_full=y_c_full, y_d_full=y_d_full, c_c=c_c, c_d=c_d) 
    
end

In [ ]:
function simulation_spaces(ae;BurnTm = 100,Tsim = BurnTm + 250*4,Nsim = 1000)
    #@unpack Nsim, Tsim = ae
    
    #######Simulation##############
    # index of variables (these variables have a grid)
    SimIndexShock  = -666*ones(Int32, Nsim,Tsim) # aggregate productivity
    SimIndexTaxRate= -666*ones(Int32, Nsim,Tsim) # tax level
    SimIndexCop = -666*ones(Int32, Nsim,Tsim) # copper

    SimDefault     = -666*ones(Int32, Nsim,Tsim) # equals to "0" if not in default, equals to "1" if economy in default (and no access to credit markets)
    SimDefaultEvent= -666*ones(Int32, Nsim,Tsim) # counts the number of defaults, SimDefault(nnn,ttt) is "1" but SimDefault(nnn,ttt-1) = 0

    SimB   = -666*ones(Nsim,Tsim) # assets/debt
    SimBIndx = -666*ones(Int32, Nsim,Tsim) # assets/debt index
    SimBs   = -666*ones(Nsim,Tsim) # assets/debt
    SimBsIndx = -666*ones(Int32, Nsim,Tsim) # assets/debt index

    # simulated variables
    SimCons     = -666*ones(Nsim,Tsim) # consumption
    SimDebt     = -666*ones(Nsim,Tsim) # debt
    SimDbtRatio = -666*ones(Nsim,Tsim) # debt/gdp ratio
    SimLabr     = -666*ones(Nsim,Tsim) # labor
    SimOutp     = -666*ones(Nsim,Tsim) # output
    SimTaxRate  = -666*ones(Nsim,Tsim) # tax rate
    SimTaxRev   = -666*ones(Nsim,Tsim) # tax revenues
    SimIntRate  = -666*ones(Nsim,Tsim) # interest rate
    SimGRatio   = -666*ones(Nsim,Tsim) # government/total output ratio
    SimGovtBce  = -666*ones(Nsim,Tsim) # government balance/output
    Simshockprod = -666*ones(Nsim,Tsim)
    Simshockprod_def = -666*ones(Nsim,Tsim)
    SimValue = -666*ones(Nsim,Tsim)
    
    SimGovt     = zeros(Nsim,Tsim) # government expenditure
    SimPrice    = zeros(Nsim,Tsim) # price of government bonds  
    SimCopRev   = zeros(Nsim,Tsim)
    SimCopProd  = zeros(Nsim,Tsim)

    SimΘ  = zeros(Nsim,Tsim)
    SimΘ1 = zeros(Nsim,Tsim)
    SimΘ2 = zeros(Nsim,Tsim)
    SimΘ3 = zeros(Nsim,Tsim)
    SimΘ4 = zeros(Nsim,Tsim)    
    
    Sim_hp = zeros(Nsim,Tsim)
    
    SimBsRatio    = zeros(Nsim,Tsim)
    Sim_escenario = zeros(Nsim,Tsim)
    SimFullOutp   =  zeros(Nsim,Tsim)
    SimBNRatio    =  zeros(Nsim,Tsim)
    SimBsRatiomean =  zeros(Nsim,Tsim)
    SimBs_escenario =  zeros(Nsim,Tsim)
    SimBs_escenario_mix = zeros(Nsim,Tsim)
    SimProb = zeros(Nsim,Tsim)
    
    return (SimIndexShock=SimIndexShock, SimIndexTaxRate=SimIndexTaxRate, 
        SimDefault=SimDefault, SimDefaultEvent=SimDefaultEvent, SimB=SimB, SimBIndx=SimBIndx,
        SimBs=SimBs,SimBsIndx=SimBsIndx, SimCons=SimCons, SimDebt=SimDebt, 
        SimOutp=SimOutp, SimTaxRate=SimTaxRate, SimDbtRatio=SimDbtRatio, SimLabr=SimLabr, 
        SimPrice=SimPrice, SimIntRate=SimIntRate, SimTaxRev=SimTaxRev, SimGovt=SimGovt,
        SimGRatio=SimGRatio, SimGovtBce=SimGovtBce, Sim_hp=Sim_hp,
        Simshockprod=Simshockprod, Simshockprod_def=Simshockprod_def, SimValue=SimValue,
        BurnTm=BurnTm, Tsim=Tsim, Nsim=Nsim, SimBsRatio=SimBsRatio, 
        SimIndexCop=SimIndexCop, SimCopRev=SimCopRev, SimCopProd=SimCopProd,
        Sim_escenario=Sim_escenario, SimFullOutp=SimFullOutp, 
        SimΘ=SimΘ, SimΘ1=SimΘ1, SimΘ2=SimΘ2, SimΘ3=SimΘ3, SimΘ4=SimΘ4, 
        SimBNRatio=SimBNRatio, SimBsRatiomean=SimBsRatiomean, SimBs_escenario=SimBs_escenario, 
        SimBs_escenario_mix=SimBs_escenario_mix, SimProb
        )
end